# Notes

PAGE: CAREER OUTCOMES:
- The companies are in an image, so our scraping method did not include them. Our RAG system is not going to know

012_%20.md  = the markdown with ALL the other markdowns, grouped in the same one (collective).


# Scraping Strategy


---

###Strategy 1: Switching from BeautifulSoup's get_text() to Markdownify
LLMs perform better when processing text from markdown rather than CSV. That is why we opted to try this strategy. For that, we knew two possible platforms: `Markdownify` and `Scraping Ant`. After some research, we decided to go with `Markdownigy`, as `Scraping Ant` was for more JavaScript-heavy websites, which we did not find to be the case with the M.S. in Applied Data Science one.

**Benefits of this approach:**
- Better semantic search (headings provide context)
- You preserve links for reference tracking
- You can keep table structure retention (this was important for the course schedule page, specifically)
- Appropriately formats lists

**Why not Beautiful Soup?** It removed all semantic HTML elements. This meant headings, lists, links, and tables became plain text, making it impossible for LLMs to understand document hierarchy.

---

### Strategy 2: Intelligent Content Extraction with Multiple Selectors
This attempts to find the main content area using multiple CSS selectors in order of preference: `<main>, <article>, <div class="content">, <div id="content">`, before falling back to `<body>`.

**Why do this?** Many websites don't use semantic HTML5 tags. Some use `<main>`, others use `<article>`. Our solution improves chances of finding actual content area rather than grabbing entire page with `nav` and `footer`.

**Benefits:**
- Reduces noise from repeated navigation menus
- Eliminates footer content on every page
- Focuses embedding vectors on actual page content, which improves search relevance, since it excludes boilerplate that would serve as noise

```bash
pythonmain_content = (
    soup.find('main') or
    soup.find('article') or
    soup.find('div', class_='content') or
    soup.find('div', id='content') or
    soup.body
)
```
---
### Strategy 3: Removing Navigation Elements and Sidebars
After identifying the main content, removes `<nav>, <footer>, <aside>`, and `<header>` from inside the main content.

**Why do this?** Even iside the `main` area, there might be some noise. It is good procedure because:
- Improves token efficiency in vector databases
- Prevents irrelevant matches (searching for "financial aid" won't match every page with a "Financial Aid" link in the sidebar)
- Creates even cleaner, more focused documents

```bash
 aside_count = len(main_content.find_all('aside'))
        for element in main_content.find_all(['nav', 'footer', 'aside', 'header']):
            print(f"Removing {element.name} tag")
            element.decompose()
        print(f"Removed {aside_count} aside tags")
```
---
### Strategy 4: URL Fragment Cleaning to Prevent Duplicates
This removes URL fragments (everything after #) before checking if a URL has been visited, preventing the same page from being scraped multiple times with different anchors.

**Why do this?** Websites often have internal links to sections: `/page#section1`, `/page#section2`, `/page#main`. Without cleaning, the scraper treats these as different URLs and scrapes the same page multiple times. This was a problem with the UChicago websie, specifically witht the `accordion-tab`. For both the in-person and online program pages, several `accordion-tab` were saved, all with the exact same content. You can see in this image:

<img src="/content/drive/MyDrive/Gen AI Shared/download.png" width="400">

For a larger-scale project, this mistake could severely increase storage and processing costs. Either way, it eliminates duplicated documents in the vector database.

``` bash
pythonfrom urllib.parse import urlparse, urlunparse

parsed = urlparse(url)
base_url = urlunparse(parsed._replace(fragment=''))

if base_url in visited:
    print(f"{'  ' * depth}  Skipping duplicate: {url}")
    return data

visited.add(base_url)
```
---
### Strategy 6: Extracting Footer Content Separately

On the first page only, extracts the footer content and saves it as a separate markdown file with a semantic name like contact_and_policies.md.

We did this because the footer has relevant information and we wanted to save it once. Instead of leaving it in every page, we saved it once.


```bash
if url == start_url and depth == 0:
    footer = soup.find('footer')
    if footer:
        footer_text = md(str(footer))
        with open('/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/contact_and_policies.md', 'w') as f:
            f.write("# Data Science Institute - Contact Information and Policies\n\n")
            f.write(footer_text)
        print("  ✓ Saved footer information separately")
```
---
### Strategy 6: Creating a Metadata Manifest
Generates a JSON file containing metadata about each scraped page including filename, URL, word count, character count, and scrape timestamp.

**Why do this?**
Without a manifest, we'd need to open each markdown file to understand what was scraped, making it difficult to track scraping coverage, debug missing content and understand the dataset structure.

More than improving RAG, this is helpful in debugging (good standards). It also helps us to figure out an error or need for improvement of the RAG retrieval (testing).

```bash
manifest = []
for idx, row in df_final.iterrows():
    manifest.append({
        'filename': filename,
        'url': row['url'],
        'word_count': len(row['text'].split()),
        'char_count': len(row['text']),
        'scraped_date': datetime.now().isoformat()
    })

with open('/content/drive/MyDrive/Gen AI Shared/manifest.json', 'w') as f:
    json.dump(manifest, f, indent=2)
```

---

### Strategy 7: Creating a Combined Document for Simple RAG (PONTENTIALLY UNNECESSARY... CHECK)
**this could be redundant, because i will be uploading the spearate files in RAG.ipynb**

In addition to individual files, creates one large markdown file containing all scraped content with clear section separators and a table of contents. Some RAG systems work better with single large documents rather than many small files. It's another option, and we'll test which one is better in our system.

```bash
combined_md = "# MS in Applied Data Science - Complete Documentation\n"
combined_md += f"*Generated: {datetime.now().strftime('%Y-%m-%d')}*\n"

# Add table of contents
for idx, row in df_final.iterrows():
    page_name = url_parts[-1] or url_parts[-2] or 'index'
    combined_md += f"- [{page_name}](#{idx:03d}-{page_name.lower()})\n"

# Add all content with clear separators
for idx, row in df_final.iterrows():
    combined_md += f"\n\n## {idx:03d}-{page_name.lower()}\n"
    combined_md += f"*Source: {row['url']}*\n\n"
    combined_md += row['text']
    combined_md += "\n\n---"
```

### Strategy 8: Configuring Markdownify for Consistency

This passes specific parameters to Markdownify to ensure consistent formatting: ATX headers (#), dash bullets (-), and asterisk emphasis (*).

**Why do this?** Default markdownify might use different styles inconsistently:
- Headers might be underlined (Setext) or prefixed with # (ATX)
- Bullets might be *, -, or +
- Emphasis might be * or _

This inconsistency makes parsing and chunking more difficult.

``` bash
text = md(
    str(main_content),
    heading_style="ATX",      # Always use # for headers
    bullets="-",              # Always use - for bullets
    strong_em_symbol="*"      # Always use * for bold/italic
)
```
---
### Strategy 9: Cleaning Excessive Whitespace
Uses regex to replace more than two consecutive newlines with exactly two newlines, cleaning up excessive whitespace that often results from HTML conversion. We didn't see specific cases in the UChicago website, but we thought it was safer to run this cleaning step.

- Divs with padding/margins
- Empty paragraphs for spacing
- Multiple nested containers

``` bash
import re

# Clean up excessive newlines (more than 2 in a row)
text = re.sub(r'\n{3,}', '\n\n', text)

# Clean up spacing around headers
text = re.sub(r'\n\n+#', '\n\n#', text)
```

---
### Strategy 10: Smart Filename Generation from URLs

Extracts meaningful filenames from URLs by taking the last path segment (or second-to-last if the last is empty) and combining with a numeric prefix for ordering. In simpler terms, appropriately name the many markdown files the code is creating, so it is more interpretable and easier to find.

``` bash
url_parts = row['url'].rstrip('/').split('/')
filename = url_parts[-1] or url_parts[-2] or 'index'
filename = f"{idx:03d}_{filename}.md"
```
---

### Strategy 11: Removing Image Tags During Processing
Identifies and removes all <img> tags from the HTML before converting to markdown. Our system is not multimodal, so keeping the images would add noise and occupy storage.

``` bash
img_count = len(main_content.find_all('img'))
for img in main_content.find_all('img'):
    img.decompose()

print(f"{'  ' * depth}  ✓ {len(text):,} chars, {img_count} images removed")
```


# Code

In [ ]:
#!pip install requests beautifulsoup4 markdownify

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin, urlparse
from google.colab import drive
from IPython.display import Markdown
import re
import json
from datetime import datetime
import os
from urllib.parse import urlparse, urlunparse

drive.mount('/content/drive')

def improved_scrape(url, start_url, depth=0, max_depth=20, visited=None, data=None):
    if visited is None:
        visited = set()
    if data is None:
        data = []

    # Clean the URL by removing fragment
    parsed = urlparse(url)
    base_url = urlunparse(parsed._replace(fragment=''))

    # Check if we've already scraped this base URL
    if base_url in visited:
        print(f"{'  ' * depth}  Skipping duplicate: {url}")
        return data

    # Check depth after duplicate check
    if depth > max_depth:
        return data

    print(f"{'  ' * depth}Scraping: {url} (base: {base_url})")
    visited.add(base_url)

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Better content extraction
        main_content = (
            soup.find('main') or
            soup.find('article') or
            soup.find('div', class_='content') or
            soup.find('div', id='content') or
            soup.body  # fallback to body, not full soup
        )

        aside_count = len(main_content.find_all('aside'))
        for element in main_content.find_all(['nav', 'footer', 'aside', 'header']):
            print(f"Removing {element.name} tag")
            element.decompose()
        print(f"Removed {aside_count} aside tags")

        extraction_method = "main_content" if main_content != soup.body else "body"

        # Remove images
        img_count = len(main_content.find_all('img'))
        for img in main_content.find_all('img'):
            img.decompose()

        # Convert to markdown
        text = md(
            str(main_content),
            heading_style="ATX",      # Use # for headers
            bullets="-",              # Consistent bullet style
            strong_em_symbol="*"      # Use * for bold/italic
        )

        # Clean whitespace/newlines
        text = re.sub(r'\n{3,}', '\n\n', text)
        text = re.sub(r'\n\n+#', '\n\n#', text)

        # Footer extraction
        if url == start_url and depth == 0:  # First page only
            footer = soup.find('footer')
            if footer:
                # Save footer separately
                footer_text = md(str(footer))
                with open('/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/footer_contact_info.md', 'w') as f:
                    f.write("# Data Science Institute - Contact Information and Policies\n\n")
                    f.write(footer_text)
                print("Saved footer information separately")


        data.append({
            'url': url,
            'text': text
        })

        print(f"{'  ' * depth}  {len(text):,} chars, {img_count} images removed, method: {extraction_method}")

        # Find and follow links
        for link_tag in soup.find_all("a", href=True):
            link = urljoin(url, link_tag["href"])

            parsed_link = urlparse(link)
            clean_link = urlunparse(parsed_link._replace(fragment=''))

            if clean_link.startswith(start_url) and clean_link not in visited:
                time.sleep(1)
                improved_scrape(clean_link, start_url, depth + 1, max_depth, visited, data)

    except Exception as e:
        print(f"{'  ' * depth}  Failed: {e}")

    return data

# Start full scraping
print("Starting improved scraping with main content extraction...")
start_url = "https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/"

all_data = improved_scrape(start_url, start_url, max_depth=20)

# Create final DataFrame
df_final = pd.DataFrame(all_data)

print(f"\nSCRAPING COMPLETE!")
print(f"Total pages scraped: {len(df_final)}")

# Create directory for markdown files
os.makedirs('/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/pages', exist_ok=True)

# Save each page as markdown and build manifest
manifest = []
for idx, row in df_final.iterrows():

    # Create clean filename from URL
    url_parts = row['url'].rstrip('/').split('/')
    filename = url_parts[-1] or url_parts[-2] or 'index'
    filename = f"{idx:03d}_{filename}.md"

    # Save markdown file
    filepath = f'/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/pages/{filename}'
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(f"# Source: {row['url']}\n")
        f.write(f"# Scraped: {datetime.now().strftime('%Y-%m-%d')}\n\n")
        f.write(row['text'])

    # Add to manifest
    manifest.append({
        'filename': filename,
        'url': row['url'],
        'word_count': len(row['text'].split()),
        'char_count': len(row['text']),
        'scraped_date': datetime.now().isoformat()
    })

    print(f"Saved: {filename} ({len(row['text'])} chars)")

# Save manifest
with open('/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/manifest.json', 'w') as f:
    json.dump(manifest, f, indent=2)
print(f"\nSaved {len(manifest)} markdown files and manifest.json")

# Create combined document for RAG
print("\nCreating combined document...")
combined_md = "# MS in Applied Data Science - Complete Documentation\n"
combined_md += f"*Generated: {datetime.now().strftime('%Y-%m-%d')}*\n"
combined_md += f"*Total pages: {len(df_final)}*\n\n"
combined_md += "## Table of Contents\n\n"

# Add table of contents
for idx, row in df_final.iterrows():
    url_parts = row['url'].rstrip('/').split('/')
    page_name = url_parts[-1] or url_parts[-2] or 'index'
    combined_md += f"- [{page_name}](#{idx:03d}-{page_name.lower()})\n"

combined_md += "\n---\n"

# Add all content
for idx, row in df_final.iterrows():
    url_parts = row['url'].rstrip('/').split('/')
    page_name = url_parts[-1] or url_parts[-2] or 'index'

    combined_md += f"\n\n## {idx:03d}-{page_name.lower()}\n"
    combined_md += f"*Source: {row['url']}*\n\n"
    combined_md += row['text']
    combined_md += "\n\n---"

# Save combined file
with open('/content/drive/MyDrive/Gen AI Shared/msads_complete.md', 'w', encoding='utf-8') as f:
    f.write(combined_md)
print(f"Created combined document: msads_complete.md ({len(combined_md)} chars)")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting improved scraping with main content extraction...
Scraping: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/ (base: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/)
Removed 0 aside tags
Saved footer information separately
  5,613 chars, 17 images removed, method: main_content
  Scraping: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/in-person-program/ (base: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/in-person-program/)
Removing aside tag
Removing header tag
Removed 1 aside tags
    32,489 chars, 0 images removed, method: main_content
    Scraping: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/ (base: https://datascience.uchicago.e

# Results

## Footer page

In [25]:
from IPython.display import Markdown, display

with open("/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/footer_contact_info.md", "r", encoding="utf-8") as md_file:
    markdown_content = md_file.read()

display(Markdown(markdown_content))

# Data Science Institute - Contact Information and Policies

[![](https://datascience.uchicago.edu/wp-content/uploads/2023/01/DSI_Logo_Color@1x.svg) DSI](https://datascience.uchicago.edu)

Data Science Institute
John Crerar Library Building
5730 South Ellis Avenue, Suite 150
Chicago IL 60637
data-science@uchicago.edu

© 2025 University of Chicago

* [About](https://datascience.uchicago.edu/about/)
* [People](https://datascience.uchicago.edu/about/leadership-staff/)
* [Research](https://datascience.uchicago.edu/research/)
* [Education](https://datascience.uchicago.edu/education/)
* [Outreach](https://datascience.uchicago.edu/outreach/)
* [News](https://datascience.uchicago.edu/news-events/news/)
* [Events](https://datascience.uchicago.edu/news-events/events/)
* [Jobs](https://datascience.uchicago.edu/about/jobs/)
* [Newsletter Archive](https://datascience.uchicago.edu/newsletter-archive/)
* [Physical Sciences Division](https://physicalsciences.uchicago.edu/)
* [Nondiscrimination Statement](https://datascience.uchicago.edu/nondiscrimination-statement/)
* [Accessibility](https://accessibility.uchicago.edu)
* [Contact](https://datascience.uchicago.edu/about/contact/)

Get Updated
-----------











* Newsletter Signup\*
* Name

  This field is for validation purposes and should be left unchanged.

Follow
------

## Others

In [31]:
for file in os.listdir("/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/pages"):
    if file.endswith(".md"):
      print()
      print()
      print()
      print()
      print(200*'=')
      print(200*'=')
      print(200*'=')
      print(f"PRINTING: {file}")
      print(200*'=')
      print(200*'=')
      print(200*'=')
      print()
      print()
      print()
      print()
      with open(os.path.join("/content/drive/MyDrive/Gen AI Shared/msads_pages_bruna/pages", file), "r", encoding="utf-8") as md_file:
          markdown_content = md_file.read()
          display(Markdown(markdown_content))





PRINTING: 000_ms-in-applied-data-science.md






# Source: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/
# Scraped: 2025-07-27

Elevate Your Expertise in Data Science

The University of Chicago’s MS in Applied Data Science program equips you with in-demand expertise and an unparalleled network of global alumni. Take the next step and start your application today.

[How to Apply](https://datascience.uchicago.edu/how-to-apply/)

## Programs

Choose from full- and part-time options in our In-Person and Online programs.

## [In-Person Program](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/in-person-program/)

If you are an early career professional or need to complete a master's in one year, the In-Person program is for you.

[Learn More](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/in-person-program/)

## [Online Program](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/online-program/%20)

If you want 360° flexibility and the same rigorous curriculum and outcomes as an in-person degree, the Online Program is for you.

[Learn More](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/online-program/%20)

## [MBA/MS Program](https://www.chicagobooth.edu/mba/joint-degree/mba-ms-applied-data-science?sc_lang=en)

The joint degree with UChicago’s Booth School of Business is ideal for ambitious students looking to supplement their MBA studies with a cutting-edge education in data science.

[Learn More](https://www.chicagobooth.edu/mba/joint-degree/mba-ms-applied-data-science?sc_lang=en)

## Get in Touch

We welcome the opportunity to [discuss your interest in our program](https://apply-psd.uchicago.edu/register/?id=ef0bc7e7-7b6a-4888-92e1-0574384e9b9c&amp), answer any questions you have, and more.

## How to Apply

You have questions; we have answers. Whether you are just beginning your search or are ready to apply, we are here to help you take the next step.

[How to Apply](https://datascience.uchicago.edu/how-to-apply/)

## Industry Leading Faculty

UChicago’s Master’s in Applied Data Science program recruits industry leaders from across the country. Meet our instructors and learn how they are shaping the landscape of data science.

[Learn More](https://datascience.uchicago.edu/about-us/)

## Data in Action - Capstone Projects

Whether you are early in your career or more advanced, you will benefit from our real-world Capstone Experience. You will have the opportunity to help top companies across multiple sectors to solve real business problems. [Sample Capstone Projects](https://datascience.uchicago.edu/capstone-projects/).

[Learn More](https://datascience.uchicago.edu/capstone-projects/)

## Start Your Application

The [application portal](https://apply-psd.uchicago.edu/apply/) to be considered for Autumn 2025 enrollment closed on June 23, 2025.

The application for entrance in 2026 will open in September 2025. [Be the first to know](https://ms-ads.datascience.uchicago.edu) when the application opens, view other key [Events & Deadlines](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/events-deadlines/).  The [Online Program](https://datascience.uchicago.edu/education/masters-programs/online-program/) will also take applications for enrollment beginning in September 2025.

[FAQs](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/)

## Related News, Insights, and Past Events

[DSI NewsJul 17, 2025

## Schmidt AI in Science Fellowship Seminar Series Highlights AI for Scientific Discovery](https://datascience.uchicago.edu/news/schmidt-ai-in-science-fellowship-seminar-series-highlights-ai-for-scientific-discovery/)
[Campus NewsJul 15, 2025

## 2025 Midwest Machine Learning Symposium Demonstrates Regional Excellence](https://datascience.uchicago.edu/news/2025-midwest-machine-learning-symposium-demonstrates-regional-excellence/)
[Applied Data ScienceJul 15, 2025

## UChicago MS-ADS Students Present Capstone Project at Midwest ML Symposium](https://datascience.uchicago.edu/news/uchicago-ms-ads-students-present-capstone-project-at-midwest-ml-symposium/)
[DSI NewsJul 11, 2025

## UChicago Alums Talk Careers in Data Science for Social Good at Parliament Data](https://datascience.uchicago.edu/news/uchicago-alums-talk-careers-in-data-science-for-social-good-at-parliament-data/)
[Jul

17

Past EventJul 17, 2025

## Info Session: Rising Stars in Data Science Workshop](https://datascience.uchicago.edu/events/info-session-rising-stars-in-data-science-workshop/)
[Applied Data ScienceJul 02, 2025

## Hackathon Unites United Airlines and UChicago MS-ADS Students to Tackle Aviation with AI](https://datascience.uchicago.edu/news/hackathon-unites-united-airlines-and-uchicago-ms-ads-students-to-tackle-aviation-with-ai/)
[Applied Data ScienceJul 01, 2025

## Hank Snowdon Took a Swing at Data Science and Landed in the MLB](https://datascience.uchicago.edu/news/hank-snowdon-took-a-swing-at-data-science-and-landed-in-the-mlb/)
[DSI NewsJun 30, 2025

## Under the Hood: The Mathematics of AI](https://datascience.uchicago.edu/news/under-the-hood-the-mathematics-of-ai/)
[Campus NewsJun 26, 2025

## Spring Distinguished Speaker Series Fosters Conversations on the Future of AI with Leaders in the Field](https://datascience.uchicago.edu/news/spring-distinguished-speaker-series-fosters-conversations-on-the-future-of-ai-with-leaders-in-the-field/)
[DSI NewsJun 20, 2025

## The DSI Welcomes Summer Students to Explore the World of Research](https://datascience.uchicago.edu/news/the-dsi-welcomes-summer-students-to-explore-the-world-of-research/)





PRINTING: 001_in-person-program.md






# Source: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/in-person-program/
# Scraped: 2025-07-27

## MS in Applied Data Science facet-arrow-down

- [Overview](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/)
- [In-Person Program](https://datascience.uchicago.edu/education/masters-programs/in-person-program/)
- [Online Program](https://datascience.uchicago.edu/education/masters-programs/online-program/)
- [Capstone Projects](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)
- [Course Progressions](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/course-progressions/)
- [How to Apply](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/how-to-apply/)
- [Events & Deadlines](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/events-deadlines/)
- [Tuition, Fees, & Aid](https://datascience.uchicago.edu/education/tuition-fees-aid/)
- [Our Students](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/our-students/)
- [Faculty, Instructors, Staff](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/)
- [FAQs](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/)
- [Explore the MS-ADS Campus](https://datascience.uchicago.edu/explore-the-ms-ads-campus/)
- [Career Outcomes](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/career-outcomes/)
- [Get In Touch](https://apply-psd.uchicago.edu/register/?id=ef0bc7e7-7b6a-4888-92e1-0574384e9b9c&amp)

## Your Career Success

Take the next step to advance your career with UChicago’s MS in Applied Data Science.

The In-Person program admits full- and part-time students for entrance in Autumn quarter annually. The full-time, in-person MS in Applied Data Science program is [STEM/OPT eligible](https://internationalaffairs.uchicago.edu/students/prospective-students). Please visit the [Online](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/online-program/) program page if you are interested in those full- and part-time options.

## Your Engagement

If you learn best in an in-person classroom environment and prefer to live in or near to Chicago, IL, the [Master’s in Applied Data Science](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/) In-Person program is ideal for you. Your high-tech classrooms are located in downtown Chicago ([NBC Tower](https://nbc-tower.com/), [Gleacher Center](https://www.gleachercenter.com/about)), and you will have access to tailored, in-person student services and program amenities. You will complete 12 courses for the MS degree and can graduate in 12-18 months [full-time](https://datascience.uchicago.edu/test-course-progressions/). [Part-time](https://datascience.uchicago.edu/test-course-progressions/) options available. Most courses are from 6-9pm Monday through Thursday with some offered on Fridays and Saturdays. This allows you to work in an internship and/or job during the program. Select courses are offered during the day. Learn more about [Tuition, Fees, & Aid](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/tuition-fees-aid/).

## Your Student Experience

As an In-Person program student, you will have access to expert [faculty and instructors](https://datascience.uchicago.edu/about-us/) with industry expertise, a full-service [student affairs team,](https://datascience.uchicago.edu/about-us/) and an unparalleled network of [global alumni](https://physicalsciences.uchicago.edu/about/alumni-friends/). [Our team](https://datascience.uchicago.edu/about-us/) is passionate about supporting a Signature Student Experience tailored to your needs.

*Program Director, [Greg Green, PhD](https://datascience.uchicago.edu/people/greg-green/)*

## Your Outcomes

Your success is our success. Graduates of UChicago’s Master’s in Applied Data Science program consistently demonstrate competitive outcomes. You will have full access to our tailored career services and external partnerships team to help you advance your career in data science–whether you are launching your career, interested in pivoting, or want to move up within your current company. You can take advantage of in-house career services advising and coaching, tailored networking events, career fairs to connect directly with employers, internship placement support, and more.

## By and For Data Science Innovators

You will earn UChicago’s Master’s in Applied Data Science by successfully completing 12 courses (6 core, 4 elective, 2 Capstone) and our tailored Career Seminar\*.

To keep up with the rapidly evolving field and job market, you will be challenged by our rigorous curriculum that is designed by and for data science innovators and leaders. Courses are reviewed annually to ensure the content keeps pace with the rapidly evolving landscape of data science.

You have the flexibility to pursue the [Master’s in Applied Data Science](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/) degree on a part- or full-time schedule. Part-time students enroll in two courses each quarter and take their courses in the evenings or on Saturdays. Full-time students take three courses per quarter. Some of their courses may be offered during the day. All courses are taught at the NBC Tower or Gleacher Center in downtown Chicago.

### Foundational Courses (noncredit, optional)

Foundational noncredit courses are designed and taught by Master’s in Applied Data Science [faculty and instructors](https://datascience.uchicago.edu/about-us/). These optional courses—available at no additional cost— provide the basis for the rigorous Applied Data Science degree. Course content undergirds the theoretical, strategic, and practical data science studies you will encounter in the rest of the curriculum.  

Beginning in academic year 2024-25, all entering students will complete a required online Foundational Skills Assessment. The assessment helps faculty and advisors understand how to best support you once you begin in the program. The four Foundational noncredit courses are listed below. Please note that Introduction to Statistical Concepts and R are considered pre-quarter courses and therefore take place during the 5 weeks leading up to your first quarter in the program. All Foundational courses are completed virtually for all students regardless of enrollment in the In-Person or Online Program.

Read More +
Show Less -

### Career Seminar (noncredit, required)

Increasingly, employers demand data scientists and analytics professionals who are not only technically excellent but also superior collaborators, reliable communicators, ethical problem-solvers, and more. To ensure that our graduates remain as top candidates in the job market, we provide tailored support through our required, multi-quarter Career Seminar. The Seminar is designed to counter-balance students’ time commitments in other courses. \*Students with significant, relevant full-time work experience may be eligible to waive this requirement.

### Core Courses (6)

You will complete six core courses toward your [Master’s in Applied Data Science](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/) degree. Core courses allow you to build your theoretical data science knowledge and practice applying this theory to examine real-world business problems.

### Elective Courses (4)

Explore advanced analytics strategies and applications. You will complete four required electives toward your 12-course degree program. We continually add electives to evolve with the data science landscape. Past electives include: Advanced Computer Vision with Deep Learning, Advanced Machine Learning and Artificial Intelligence, Bayesian Methods, Data Science for Algorithmic Marketing, Data Visualization Techniques, Digital Marketing Analytics in Theory and Practice, Financial Analytics, Health Analytics, Machine Learning Operations, Natural Language Processing and Cognitive Computing, Real Time Intelligent Systems, Reinforcement Learning, Supply Chain Optimization.

### Capstone (2)

The required [Capstone Project](https://datascience.uchicago.edu/capstone-projects/) is completed over two quarters and covers research design, implementation, and writing. Full-time students start their Capstone Project in their third quarter. Part-time students generally begin the Capstone Project two quarters before their projected graduation quarter. Students choose among industry- and research-focused projects.

### Get in Touch

If you would like to receive periodic information about the MS in Applied Data Science program, [please enter your information in this form](https://apply-psd.uchicago.edu/register/?id=ef0bc7e7-7b6a-4888-92e1-0574384e9b9c&amp). Visit us on [YouTube](https://www.youtube.com/channel/UCf_vMLzedhxTSvsUwVgpAgA) and [GitHub](https://github.com/uchicago-dsi).

- [Noncredit Courses](#) 

  - [Career Seminar (Seminar, required)](#accordion-tab-0) 

    The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.
  - [Introduction to Statistical Concepts (Foundational, optional)](#accordion-tab-1) 

    This course is held in the five weeks leading up to the start of your first quarter and provides general exposure to basic statistical concepts necessary for success in advanced courses in the program. 0 units, no cost.
  - [R for Data Science (Foundational, optional)](#accordion-tab-2) 

    This course is held the five weeks leading up to the start of your first quarter and is an introduction to the essential concepts and techniques for the statistical computing language R. 0 units, no cost.
  - [Python for Data Science (Foundational, optional)](#accordion-tab-3) 

    This course is held concurrently with the first five weeks of your first quarter in the program and starts with an introduction to the Python programming language basic syntax and environment. 0 units, no cost.
  - [Advanced Linear Algebra for Machine Learning (Foundational, optional)](#accordion-tab-4) 

    This course is held concurrently with the second five weeks of your first quarter in the program and is focused on the theoretical concepts and real-life applications of linear algebra for machine learning. 0 units, no cost.
  - [Brush up on the Basics (Optional resource)](#accordion-tab-5) 

    If you would like to gauge your preparation in Foundational course topics, we recommend specific Coursera courses that cover very similar topics.

    If you would like to gauge your preparation in Foundational course topics, we recommend specific Coursera courses that cover very similar topics.

    Four Coursera courses cover very similar topics. You can review the Coursera curricula to see if you are already well-prepared, or if you like, study their materials to brush up on some or all of these topics.

    [Mathematics for Machine Learning: Linear Algebra (offered by University College London)](https://www.coursera.org/learn/linear-algebra-machine-learning)

    [Basis Data Descriptors, Statistical Distributions, and Application to Business Decisions (offered by Rice University)](https://www.coursera.org/learn/descriptive-statistics-statistical-distributions-business-application?specialization=business-statistics-analysis)

    [Python for Data Science, AI, & Development (offered by IBM)](https://www.coursera.org/learn/python-for-applied-data-science-ai?specialization=ibm-data-science)

    [Data Analysis with R Programming (offered by Google)](https://www.coursera.org/learn/data-analysis-r)
- [Core Courses](#)

  - [Time Series Analysis and Forecasting](#accordion-tab-0) 

    Time Series Analysis is a science as well as the art of making rational predictions based on previous records. It is widely used in various fields in today’s business settings.
  - [Statistical Models for Data Science](#accordion-tab-1) 

    In a traditional linear model, the observed response follows a normal distribution, and the expected response value is a linear combination of the predictors.  Since Carl Friedrich Gauss (1777-1855) and Adrien-Marie Legendre (1752-1833) created this linear model framework in the early 1800s, the “Linear Normal” assumption has been the norm in statistics/data science for almost two centuries.  New methods based on probability distributions other than Gaussian appeared only in the second half of the twentieth century. These methods allowed working with variables that span a broader variety of domains and probability distributions. Besides, methods for the analysis of general associations were developed that are different from the Pearson correlation.
  - [Machine Learning I](#accordion-tab-2) 

    This course is aimed at providing students an introduction to machine learning with data mining techniques and algorithms. It gives a rigorous methodological foundation in analytical and software tools to successfully undertake projects in Data Science. Students are exposed to concepts of exploratory analyses for uncovering and detecting patterns in multivariate data, hypothesizing and detecting relationships among variables, conducting confirmatory analyses, and building models for predictive and descriptive purposes. It will present predictive modeling in the context of balancing predictive and descriptive accuracies.
  - [Machine Learning II](#accordion-tab-3) 

    The objective of this course is three-folds–first, to extend student understanding of predictive modeling with machine learning concepts and methodologies from Machine Learning 1 into the realm of Deep Learning and Generative AI. Second, to develop the ability to apply those concepts and methodologies to diverse practical applications, evaluate the results and recommend the next best action. Third, to discuss and understand state-of-the machine learning and deep learning research and development and their applications.
  - [Data Engineering Platforms for Analytics or Big Data and Cloud Computing](#accordion-tab-4) 

    Data Engineering Platforms teaches effective data engineering—an essential first step in building an analytics-driven competitive advantage in the market.

    Big Data and Cloud Computing teaches students how to approach big data and large-scale machine learning applications. There is no single definition of big data and multiple emerging software packages exist to work with it, and we will cover the most popular approaches.
  - [Leadership and Consulting for Data Science](#accordion-tab-5) 

    The Leadership and Consulting for Data Scientist course is focused on:

    • Learning techniques and proven methods to effectively grasp the business domain including organizational dynamics of consultancies and client organizations  
    • Developing relevant solutions to enterprise problems using the sampling methods, traditional statistical techniques and modern machine learning models that deliver value to the organization  
    • Practicing successful project delivery through effective data discovery, influential team membership and leadership, project management, and communication at every stage

    This course will not only make you a better data scientist; it will make you and your analyses more approachable, more persuasive, and ultimately more successful.
  - [Data Science Capstone Project](#accordion-tab-6) 

    The required Capstone Project is completed over two quarters and covers research design, implementation, and writing. Full-time students start their capstone project in their third quarter. Part-time students generally begin the capstone project in their fifth quarter.
- [Sample Elective Courses](#)

  - [Advanced Computer Vision with Deep Learning](#accordion-tab-0) 

    Computer vision is the field of computer science that focuses on creating digital systems that can process, analyze, and make sense of visual data in the same way that humans do. Deep learning is a subset of machine learning and a branch of Artificial Intelligence (AI). It involves the training, deployment, and application of large complex neural network architectures to solve cutting-edge problems. Deep Learning has become the primary approach for solving cognitive problems such as Computer Vision and Natural Language Processing (NLP) and has had a massive impact on various industries such as healthcare, retail, automotive, industrial automation, and agriculture. This course will enable students to build Deep Learning models and apply them to computer vision tasks such as object recognition, detection, and segmentation. Students will gain an in-depth understanding of the Deep Learning model development process, tools, and frameworks. Although the focus of the course will primarily be computer vision, students will work on both image and nonimage datasets during class exercises and assignments. Students will gain hands-on experience in popular libraries such as Tensorflow, Keras, and PyTorch. Students will also learn to apply state of the art models such as ResNet, EfficientNet, RCNNs, YOLO, Vision Transformers, etc. for computer vision and work on datasets such as CIFAR, ImageNet, MS COCO, and MPII Human Poses.
  - [Advanced Machine Learning and Artificial Intelligence](#accordion-tab-1) 

    Since the era of big data started, challenges associated with data analysis have grown significantly in different directions: First, the technological infrastructure had to be developed that can hold and process large amounts of data from different sources and of multiple not always well formalized formats. Second, data analysis methods had to be reviewed, selected and modified to work in distributed computational environments like combinations of in-house clusters of servers and cloud. But the biggest challenge of all is learning to think differently in order to ask new types of questions that could not be answered by analyses of less complex data streams with less complex technological infrastructure. In recent years significant progress has been achieved in creating technological ecosystems for big data analysis. Innovative technologies such as open source projects MapReduce, Hadoop, Spark, Storm, Kafka, TensorFlow, H2O, etc. allowed us to look at depths of data unseen before. We now have a growing number of sources and educational courses introducing these new tools. However, developing new data analysis methods appropriate to these new data ecosystems is more difficult than it appears.
  - [Bayesian Machine Learning with Generative AI Applications](#accordion-tab-2) 

    This course provides a strong theoretical and practical skillset for probabilistic machine learning applications. Bayesian inference and modeling methods are important for several areas including prediction, decision making, and risk assessment where modeling the uncertainty is needed. The course begins with an introduction to Bayesian statistical analysis, covering the foundations of Bayesian inference and the application of Bayes’ theorem for statistical inference. We then introduce Bayesian networks, which offer a powerful graphical tool for modeling complex systems and making probabilistic inferences. The course then advances to cover more sophisticated topics such as Markov Chain Monte Carlo (MCMC) methods for sampling from complex probability distributions, hierarchical models, and model selection techniques. The final three weeks are dedicated to cutting-edge methodologies like Generative Deep Learning, Variational Autoencoders, and Bayesian Neural Networks, all rooted in Bayesian Machine Learning. Upon completion, students will be equipped to apply Bayesian methods to a wide range of real-world problems in fields such as engineering, business, finance, and public policy, addressing challenges like missing data or training AI models that are able to say ‘I don’t know’.
  - [Data Science for Algorithmic Marketing](#accordion-tab-3) 

    This course focuses on marketing science methods and algorithms for undertaking competitive analysis in the digital landscape: market segmentation, mining databases for effective digital marketing, design of new digital and traditional products, forecasting sales and product diffusion, real time product positioning, intra omni-channel optimization and inter omni-channel resource allocation, and pricing across both omni-channel marketing effectiveness and ROI. The course will use a combination of lecture, in-class discussions, group assignments, and a final group project. The course lays special emphasis on algorithms. Hence it draws heavily from the fields of optimization, machine-learning based recommendation systems, association rules, consumer choice models, Bayesian estimation, experimentation and analysis of covariance, advanced visualization techniques for mapping brand perceptions, and analysis of social media data using advanced NLP techniques.
  - [Data Visualization Techniques](#accordion-tab-4) 

    In today’s data driven enterprise, data storytelling using effective visualization strategies is an essential skill for analytics practitioners in almost every field to explore and present data. This course focuses on modern data visualization technologies, tools, and techniques to convert raw data into actionable information. Modern data visualization tools are at the forefront of the “self-service analytics” architectures which are decentralizing analytics and breaking down IT bottlenecks for business experts. Moreover, with its foundations rooted in statistics, psychology, and computer science, data visualization shows you how to better understand the data, present clear evidence of your findings to your intended audience and tell engaging data stories through charts and graphics. This course is designed to introduce data visualization as a medium of effective communication using strategic storytelling, and the basis for interactive information dashboards.
  - [Digital Marketing Analytics in Theory and Practice](#accordion-tab-5) 

    Successfully marketing brands today requires a well-balanced blend of art and science. This course introduces students to the science of web analytics while casting a keen eye toward the artful use of numbers found in the digital space. The goal is to provide marketers with the foundation needed to apply data analytics to real-world challenges they confront daily in their professional lives. Students will learn to identify the web analytic tool right for their specific needs; understand valid and reliable ways to collect, analyze, and visualize data from the web; and utilize data in decision making for their agencies, organizations or clients. By completing this course, students will gain an understanding of the motivations behind data collection and analysis methods used by marketing professionals; learn to evaluate and choose appropriate web analytics tools and techniques; understand frameworks and approaches to measuring consumers’ digital actions; earn familiarity with the unique measurement opportunities and challenges presented by New Media; gain hands-on, working knowledge of a step-by-step approach to planning, collecting, analyzing, and reporting data; utilize tools to collect data using today’s most important online techniques: performing bulk downloads, tapping APIs, and scraping webpages; and understand approaches to visualizing data effectively.
  - [Data Science for Healthcare](#accordion-tab-6) 

    Given the breadth of the field of health analytics, this course will provide an overview of the development and rapid expansion of analytics in healthcare, major and emerging topical areas, and current issues related to research methods to improve human health. We will cover such topics as security concerns unique to the field, research design strategies, and the integration of epidemiologic and quality improvement methodologies to operationalize data for continuous improvement. Students will be introduced to the application of predictive analytics to healthcare. Students will understand factors impacting the delivery of quality and safe patient care and the application of data-driven methods to improve care at the healthcare system level, design approaches to answering a research question at the population level, become familiar with the application of data analytics to impacting care at the provider level through Clinical Decision Systems, and understand the process of a Clinical Trial.
  - [Financial Analytics](#accordion-tab-7) 

    This course concentrates on the following topics: review of financial markets and assets traded on them; main characteristics of financial analytics: returns, yields, volatility; review of stochastic models of market price and their statistical representations; concept of arbitrage, elements of arbitrage pricing approach; principles of volatility analyses, implied vs. realized volatility; correlation, cointegration and other relationships between various financial assets; market risk analytics and management of portfolios of financial assets. The course puts special emphasis on covering main steps of building analytics from visualizing data and building intuition about their structure and patterns to selecting appropriate statistical method to interpretation of the results and building analytical models. Topics are illustrated by data analysis projects using R. Basic familiarity with R is a requirement.
  - [Generative AI Principles](#accordion-tab-8) 

    This course dives into the realm of Generative AI, offering a comprehensive look into the world of Large Language Models (LLMs), image generation techniques, and the fusion of vision and text through multimodal models. Drawing from core concepts in neural networks, transformers, and advanced techniques such as prompt engineering, vision prompting, and multimodality representation, students will explore the capabilities, applications, and ethical considerations of generative models. This course culminates in hands-on projects, allowing participants to apply theory to practical scenarios.
  - [Machine Learning Operations](#accordion-tab-9) 

    The objective of this course is two-fold: first, to understand what Machine Learning Operations (MLOps) is and why it is a key component in enterprise production deployment of machine learning projects, and second, to expose students to software engineering, model engineering and state-of-the-art deployment engineering with hands-on platform and tools experience. This course crosses the chasm that separates machine learning projects/experiments and enterprise production deployment. It covers three pillars in MLOps: software engineering such as software architecture, Continuous Integration/Continuous Delivery and data versioning; model engineering such as AutoML and A/B experimentation; and deployment engineering such as docker containers and model monitoring. The course focuses on best practices in the industry that are critical to enterprise production deployment of machine learning projects. Having completed this course, a student understands the machine learning lifecycle and what it takes to go from ideation to operationalization in an enterprise environment. Furthermore, students get exposure to state-of-the-art MLOps platforms such as allegro, xpresso, Dataiku, LityxIQ, DataRobot, AWS Sagemaker, and technologies such as gitHub, Jenkins, slack, docker, and kubernetes.
  - [Natural Language Processing and Cognitive Computing](#accordion-tab-10) 

    Extracting actionable insights from unstructured text and designing cognitive applications have become significant areas of application for analytics. Students in this course will learn foundations of natural language processing, including: concept extraction; text summarization and topic modeling; part of speech tagging; named entity recognition; semantic roles and sentiment analysis. For advanced NLP applications, we will focus on feature extraction from unstructured text, including word and paragraph embedding and representing words and paragraphs as vectors. For cognitive analytics section of the course, students will practice designing question answering systems with intent classification, semantic knowledge extraction and reasoning under uncertainty. Students will gain hands-on expertise applying Python for text analysis tasks, as well as practice with multiple IBM Watson services, including: Watson Discovery, Watson Conversation, Watson Natural Language Classification and Watson Natural Language Understanding.
  - [Real Time Intelligent Systems](#accordion-tab-11) 

    Developing end-to-end automation and intelligent systems is now the most advanced area of application for analytics. Building such systems requires proficiency in programming, understanding of computer systems, as well as knowledge of related analytical methodologies, which are the skills that this course aims to teach to students. The course focuses on python and is tailored for students with basic programming knowledge in python. The course is partially project based. During the first three sessions, we will review basic python concepts and then learn more advanced python and the ways to use python to handle large data flows. The later sessions are project based and will focus on developing end-to-end analytical solutions in the following areas: Finance and trading, blockchains and crypto-currencies, image recognition, and video surveillance systems.
  - [Reinforcement Learning](#accordion-tab-12) 

    This course is an introduction to reinforcement learning, also known as neuro-dynamic programming. It discusses basic and advanced concepts in reinforcement learning and provides several practical applications. Reinforcement learning refers to a system or agent interacting with an environment and learning how to behave optimally in such an environment. An environment typically includes time, actions, states, uncertainty and rewards. Reinforcement learning combines neural networks and dynamic programming to find an optimal behavior or policy of the system or agent in a complex environment setting. Neural network approximations are used to circumvent the well-known ‘curse of dimensionality’ which has been a barrier to solving many practical applications. Dynamic programming is the key learning mechanism that the system or the agent uses to interact with the environment and improve its performance. Students will master key learning techniques and will become proficient in applying these techniques to complex stochastic decision processes and intelligent control.
  - [Supply Chain Optimization](#accordion-tab-13) 

    “Big Data” continues to grow exponentially in our large-scale transactional world where 100,000s of SKUs and millions of customers are interacting with 1:1 offers that include differential pricing, shipping timing/costs and even made to order “custom” product configurations. These consumer behaviors are quickly advancing the availability of new data and techniques within the discipline of Data Science. This elective course will give students the opportunity to apply their skills in data visualization, data mining tools, predictive modeling, and advanced optimization techniques to address Supply Chain challenges. The course focuses on the use of Advanced Predictive Modeling, Machine Learning, AI and other Data Science insight and activation tools to automate and optimize the performance of the Supply Chain. Students will also learn how to optimize the performance of the Supply Chain from the lens of multiple related disciplines including: Sales Forecasting, Warehousing/Inventory Management, Promotion, Pricing, Logistics Network Optimization, Freight Cost Management, Manufacturing, Retail POS Information, Ecommerce, Consumer Data, and Product Design/Packaging. After completing this course, you will be prepared to work in any of the numerous specialty areas possible in the world of Supply Chain Management.





PRINTING: 002_capstone-projects.md






...

# Source: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/online-program/
# Scraped: 2025-07-27

## MS in Applied Data Science facet-arrow-down

- [Overview](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/)
- [In-Person Program](https://datascience.uchicago.edu/education/masters-programs/in-person-program/)
- [Online Program](https://datascience.uchicago.edu/education/masters-programs/online-program/)
- [Capstone Projects](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)
- [Course Progressions](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/course-progressions/)
- [How to Apply](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/how-to-apply/)
- [Events & Deadlines](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/events-deadlines/)
- [Tuition, Fees, & Aid](https://datascience.uchicago.edu/education/tuition-fees-aid/)
- [Our Students](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/our-students/)
- [Faculty, Instructors, Staff](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/)
- [FAQs](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/)
- [Explore the MS-ADS Campus](https://datascience.uchicago.edu/explore-the-ms-ads-campus/)
- [Career Outcomes](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/career-outcomes/)
- [Get In Touch](https://apply-psd.uchicago.edu/register/?id=ef0bc7e7-7b6a-4888-92e1-0574384e9b9c&amp)

## Rigor Meets Flexibility

You will benefit from [the same top faculty, bright peer group, and academic rigor](https://www.youtube.com/watch?v=9kHtZxp1RVg&list=PL0IrIAIuK93EonLgPKZ7oIcpt0p_j58vm&index=9) as the In-Person program. You may complete the program full- or part-time. Full-time and part-time students begin in [Autumn quarter.](https://www.uchicago.edu/en/education-and-research/academic-calendar) Please visit the [In-Person](https://datascience.uchicago.edu/education/masters-programs/in-person-program/) program page if you are interested in exploring those full- and part-time options for entrance Autumn quarter.

*Online Program Director, [Arnab Bose, PhD](https://datascience.uchicago.edu/people/arnab-bose-phd/)*

## Master Data Science, Master Your Career

Get ready for what’s next in data science with rigorous classes, expert instructors, leading-edge technology, and an [unparalleled network of industry professionals](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/). A [full-service team](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/) dedicated to the MS in Applied Data Science program are here to help you reach your career potential as a member of the University of Chicago community.

## Get in Touch

Loading…

Read More +
Show Less -

## Your Outcomes

You will be a top competitor among data science jobs and career paths as a result of this program. You will have access to a dedicated team of faculty and staff who are committed to helping you reach your potential–whether it’s landing a key internship, preparing for a promotion, or moving into your next job, the MS in Applied Data Science program is here to help. Among other resources, you will have access to tailored, in-house career services, academic advising, and opportunities to grow your network in person and virtually.

## Your Student Experience

There are unique opportunities to support you as an Online master’s student. You will have access to the same full-service team of student affairs, career services, and instructional services experts. Jointly with [faculty and instructors](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/), our Master’s in Applied Data Science team is committed to providing a supportive and enriching student experience. Our team is passionate about supporting students engaging in online education who might also be working full-time like you. Staff who lead our Online program have a background in instructional design and best practices for online learning.

## Your Access to Alumni

As an Online student, you will have access to program alumni through Immersion Weekend events and virtual career services programs–all of which broadens your network and supports your career advancement goals. Upon graduation, you will also join a prestigious group of [Physical Sciences Division](https://physicalsciences.uchicago.edu/about/alumni-friends/) and [UChicago alumni](https://alumniandfriends.uchicago.edu/s/directory) who live and work all over the world. Our Alumni Board that plans events and programs in Chicago and other U.S. cities.

## Your Time, Your Advancement

You will engage in weekly, synchronous (real-time) classes with your instructors and peers. Live lectures provide the right forum to ask questions, work with peers, and maximize understanding of key concepts.  Collaborative and interactive learning with your classmates is facilitated by advanced meeting technology during and beyond synchronous classes.

Synchronous classes are scheduled on weekday evenings at 6 pm, CT and on occasional Saturdays at 9 am, CT. Students admitted into the online program are expected to attend the live class sessions.

Each week also includes asynchronous content (learning at your own pace) to complete when it is convenient for you. These activities prepare you for synchronous classes through lecture videos, readings, and discussions. Flexibility allows you to work on data science projects, group collaborations, and assignments on your schedule.

## In-Person ‘Immersion Weekend’

You will have the opportunity to attend a unique, in-person ‘Immersion Weekend’ in Chicago in the Autumn quarter.

**During the Immersion Weekend, you will have the opportunity to**

- Network with industry leaders and [alumni](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/our-students/)
- Meet a vibrant and supportive community of like-minded students and faculty
- Get to know, bond, and network with peers from across the country
- Participate in interactive sessions, working to develop industry solutions
- Gain career insights and discuss the latest trends and challenges in AI/ML
- Hear from students about their experience in [Capstone Projects](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)
- Tour the [Data Science Institute](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)

**Future Immersion Weekend dates are:**

- Autumn 2025 – Friday, September 26 – Saturday, September 27

We look forward to seeing you at the event!

#### Note for International Students

If you are coming from a non-US location for Immersion Weekend programming, you are encouraged to come to the U.S. in visitor/tourist status; using a B-1/B-2 visa or ESTA clearance depending on whether you hold citizenship from a Visa Waiver Country or not.

The immersion days are optional for students and do not carry any course enrollment for the time you are in Chicago. Please be clear when entering the U.S. that while you are a student in an online program at the University of Chicago, your visit for immersion weekend is not to attend credit-bearing classes, but to meet classmates and as a result you are coming to the U.S. as a visitor.

Read More +
Show Less -

## Curriculum

You will earn UChicago’s Master’s in Applied Data Science by successfully completing 12 courses (6 core, 4 elective, 2 Capstone) and our tailored Career Seminar\*.

Our rigorous curriculum is designed by and for data science innovators and leaders. Courses are reviewed annually to ensure the content keeps pace with the [rapidly evolving landscape of data science](https://datascience.uchicago.edu/news-events/news/).

You have the flexibility to pursue the [Master’s in Applied Data Science](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/) degree on a part- or full-time schedule. Part-time students enroll in two courses each quarter and take their courses in the evenings or on Saturdays. Full-time students take three courses per quarter. Some of their courses may be offered during the day.

### Foundational Courses (noncredit, optional)

Foundational noncredit courses are designed and taught by Master’s in Applied Data Science [faculty and instructors](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/). These optional courses—available at no additional cost— provide the basis for the rigorous Applied Data Science degree. Course content undergirds the theoretical, strategic, and practical data science studies you will encounter in the rest of the curriculum.  

Beginning in academic year 2024-25, all entering students will complete a required online Foundational Skills Assessment. This required assessment helps faculty and advisors understand how to best support you once you begin in the program. You may choose to opt-out of taking the Foundational Skill Assessments and instead register for all four Foundational courses. The four Foundational noncredit courses are listed below. Please note that Introduction to Statistical Concepts and R are considered pre-quarter courses and therefore take place during the 5 weeks leading up to your first quarter in the program. All Foundational courses are completed virtually for all students regardless of enrollment in the In-Person or Online Program.

Read More +
Show Less -

### Career Seminar (noncredit, required)

Increasingly, employers demand data scientists and analytics professionals who are not only technically excellent but also superior collaborators, reliable communicators, ethical problem-solvers, and more. To ensure that our graduates remain as top candidates in the job market, we provide tailored support through our required, multi-quarter Career Seminar. The Seminar is designed to counter-balance students’ time commitments in other courses. \*Students with significant, relevant full-time work experience may be eligible to waive this requirement.

### Core Courses (6)

You will complete 6 core courses toward your [Master’s in Applied Data Science](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/) degree. Core courses allow you to build your theoretical data science knowledge and practice applying this theory to examine real-world business problems.

### Elective Courses (4)

You will complete 4 required electives toward your 12-course degree program. We continually add electives to evolve with the data science landscape. Past electives include Advanced Computer Vision with Deep Learning, Advanced Machine Learning and Artificial Intelligence, Bayesian Methods, Data Science for Algorithmic Marketing, Data Visualization Techniques, Digital Marketing Analytics in Theory and Practice, Financial Analytics, Health Analytics, Machine Learning Operations, Natural Language Processing and Cognitive Computing, Real Time Intelligent Systems, Reinforcement Learning, Supply Chain Optimization.

### Capstone (2)

The required [Capstone Project](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/) is completed over 2 quarters and covers research design, implementation, and writing. Full-time students start their [Capstone Project](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/) in their third quarter. Part-time students generally begin the [Capstone Project](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/) two quarters before their projected [graduation](https://convocation.uchicago.edu/) quarter. Students choose among industry- and research-focused projects.

### Additional Resources

- [Upcoming Events & Deadlines](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/events-deadlines/)
- [FAQs](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/)
- [Tuition, Fees, & Aid](https://datascience.uchicago.edu/education/tuition-fees-aid/)

- [Noncredit Courses](#)

  - [Career Seminar (Seminar, required)](#accordion-tab-0) 

    The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.
  - [Introduction to Statistical Concepts (Foundational, optional)](#accordion-tab-1) 

    This Foundational, optional course is held in the 5 weeks leading up to the start of your first quarter. This course provides general exposure to basic statistical concepts that are necessary for students to understand the content presented in more advanced courses in the program. 0 units, no cost.
  - [R for Data Science (Foundational, optional)](#accordion-tab-2) 

    This Foundational, optional course is held in the 5 weeks leading up to the start of your first quarter. This course is an introduction to the essential concepts and techniques for the statistical computing language R. 0 units, no cost.
  - [Python for Data Science (Foundational, optional)](#accordion-tab-3) 

    This Foundational, optional course is held concurrently with the first five weeks of your first quarter in the program. This course in Python starts with an introduction to the Python programming language basic syntax and environment. 0 units, no cost.
  - [Advanced Linear Algebra for Machine Learning (Foundational, optional)](#accordion-tab-4) 

    This Foundational, optional course is held concurrently with the second five weeks of your first quarter in the program. The advanced linear algebra course is focused on the theoretical concepts and real-life applications of linear algebra for machine learning. 0 units, no cost.
  - [Brush up on the Basics (Optional resource)](#accordion-tab-5) 

    If you would like to gauge your preparation in these Foundational course topics, we recommend specific Coursera courses that cover very similar topics.

    We have identified four Coursera courses which cover very similar topics. You can review the Coursera curricula to see if you are already well-prepared, or if you like, study their materials to brush up on some or all of these topics.

    [Mathematics for Machine Learning: Linear Algebra (offered by University College London)](https://www.coursera.org/learn/linear-algebra-machine-learning)

    [Basis Data Descriptors, Statistical Distributions, and Application to Business Decisions (offered by Rice University)](https://www.coursera.org/learn/descriptive-statistics-statistical-distributions-business-application?specialization=business-statistics-analysis)

    [Python for Data Science, AI, & Development (offered by IBM)](https://www.coursera.org/learn/python-for-applied-data-science-ai?specialization=ibm-data-science)

    [Data Analysis with R Programming (offered by Google)](https://www.coursera.org/learn/data-analysis-r)
- [Core Courses](#)

  - [Time Series Analysis and Forecasting](#accordion-tab-0) 

    Time Series Analysis is a science as well as the art of making rational predictions based on previous records. It is widely used in various fields in today’s business settings.
  - [Statistical Models for Data Science](#accordion-tab-1) 

    In a traditional linear model, the observed response follows a normal distribution, and the expected response value is a linear combination of the predictors.  Since Carl Friedrich Gauss (1777-1855) and Adrien-Marie Legendre (1752-1833) created this linear model framework in the early 1800s, the “Linear Normal” assumption has been the norm in statistics/data science for almost two centuries.  New methods based on probability distributions other than Gaussian appeared only in the second half of the twentieth century. These methods allowed working with variables that span a broader variety of domains and probability distributions. Besides, methods for the analysis of general associations were developed that are different from the Pearson correlation.
  - [Machine Learning I](#accordion-tab-2) 

    This course is aimed at providing students an introduction to machine learning with data mining techniques and algorithms. It gives a rigorous methodological foundation in analytical and software tools to successfully undertake projects in Data Science. Students are exposed to concepts of exploratory analyses for uncovering and detecting patterns in multivariate data, hypothesizing and detecting relationships among variables, conducting confirmatory analyses, and building models for predictive and descriptive purposes. It will present predictive modeling in the context of balancing predictive and descriptive accuracies.
  - [Machine Learning II](#accordion-tab-3) 

    The objective of this course is three-folds–first, to extend student understanding of predictive modeling with machine learning concepts and methodologies from Machine Learning 1 into the realm of Deep Learning and Generative AI. Second, to develop the ability to apply those concepts and methodologies to diverse practical applications, evaluate the results and recommend the next best action. Third, to discuss and understand state-of-the machine learning and deep learning research and development and their applications.
  - [Data Engineering Platforms for Analytics or Big Data and Cloud Computing](#accordion-tab-4) 

    Data Engineering Platforms teaches effective data engineering—an essential first step in building an analytics-driven competitive advantage in the market.

    Big Data and Cloud Computing teaches students how to approach big data and large-scale machine learning applications. There is no single definition of big data and multiple emerging software packages exist to work with it, and we will cover the most popular approaches.
  - [Leadership and Consulting for Data Science](#accordion-tab-5) 

    The Leadership and Consulting for Data Scientist course is focused on:

    • Learning techniques and proven methods to effectively grasp the business domain including organizational dynamics of consultancies and client organizations  
    • Developing relevant solutions to enterprise problems using the sampling methods, traditional statistical techniques and modern machine learning models that deliver value to the organization  
    • Practicing successful project delivery through effective data discovery, influential team membership and leadership, project management, and communication at every stage

    This course will not only make you a better data scientist; it will make you and your analyses more approachable, more persuasive, and ultimately more successful.
  - [Data Science Capstone Project](#accordion-tab-6) 

    The required Capstone Project is completed over two quarters and covers research design, implementation, and writing. Full-time students start their capstone project in their third quarter. Part-time students generally begin the capstone project in their fifth quarter.
- [Sample Elective Courses](#)

  - [Advanced Computer Vision with Deep Learning](#accordion-tab-0) 

    Computer vision is the field of computer science that focuses on creating digital systems that can process, analyze, and make sense of visual data in the same way that humans do. Deep learning is a subset of machine learning and a branch of Artificial Intelligence (AI). It involves the training, deployment, and application of large complex neural network architectures to solve cutting-edge problems. Deep Learning has become the primary approach for solving cognitive problems such as Computer Vision and Natural Language Processing (NLP) and has had a massive impact on various industries such as healthcare, retail, automotive, industrial automation, and agriculture. This course will enable students to build Deep Learning models and apply them to computer vision tasks such as object recognition, detection, and segmentation. Students will gain an in-depth understanding of the Deep Learning model development process, tools, and frameworks. Although the focus of the course will primarily be computer vision, students will work on both image and nonimage datasets during class exercises and assignments. Students will gain hands-on experience in popular libraries such as Tensorflow, Keras, and PyTorch. Students will also learn to apply state of the art models such as ResNet, EfficientNet, RCNNs, YOLO, Vision Transformers, etc. for computer vision and work on datasets such as CIFAR, ImageNet, MS COCO, and MPII Human Poses.
  - [Advanced Machine Learning and Artificial Intelligence](#accordion-tab-1) 

    Since the era of big data started, challenges associated with data analysis have grown significantly in different directions: First, the technological infrastructure had to be developed that can hold and process large amounts of data from different sources and of multiple not always well formalized formats. Second, data analysis methods had to be reviewed, selected and modified to work in distributed computational environments like combinations of in-house clusters of servers and cloud. But the biggest challenge of all is learning to think differently in order to ask new types of questions that could not be answered by analyses of less complex data streams with less complex technological infrastructure. In recent years significant progress has been achieved in creating technological ecosystems for big data analysis. Innovative technologies such as open source projects MapReduce, Hadoop, Spark, Storm, Kafka, TensorFlow, H2O, etc. allowed us to look at depths of data unseen before. We now have a growing number of sources and educational courses introducing these new tools. However, developing new data analysis methods appropriate to these new data ecosystems is more difficult than it appears.
  - [Bayesian Machine Learning with Generative AI Applications](#accordion-tab-2) 

    This course provides a strong theoretical and practical skillset for probabilistic machine learning applications. Bayesian inference and modeling methods are important for several areas including prediction, decision making, and risk assessment where modeling the uncertainty is needed. The course begins with an introduction to Bayesian statistical analysis, covering the foundations of Bayesian inference and the application of Bayes’ theorem for statistical inference. We then introduce Bayesian networks, which offer a powerful graphical tool for modeling complex systems and making probabilistic inferences. The course then advances to cover more sophisticated topics such as Markov Chain Monte Carlo (MCMC) methods for sampling from complex probability distributions, hierarchical models, and model selection techniques. The final three weeks are dedicated to cutting-edge methodologies like Generative Deep Learning, Variational Autoencoders, and Bayesian Neural Networks, all rooted in Bayesian Machine Learning. Upon completion, students will be equipped to apply Bayesian methods to a wide range of real-world problems in fields such as engineering, business, finance, and public policy, addressing challenges like missing data or training AI models that are able to say ‘I don’t know’.
  - [Data Science for Algorithmic Marketing](#accordion-tab-3) 

    This course focuses on marketing science methods and algorithms for undertaking competitive analysis in the digital landscape: market segmentation, mining databases for effective digital marketing, design of new digital and traditional products, forecasting sales and product diffusion, real time product positioning, intra omni-channel optimization and inter omni-channel resource allocation, and pricing across both omni-channel marketing effectiveness and ROI. The course will use a combination of lecture, in-class discussions, group assignments, and a final group project. The course lays special emphasis on algorithms. Hence it draws heavily from the fields of optimization, machine-learning based recommendation systems, association rules, consumer choice models, Bayesian estimation, experimentation and analysis of covariance, advanced visualization techniques for mapping brand perceptions, and analysis of social media data using advanced NLP techniques.
  - [Data Science for Healthcare](#accordion-tab-4) 

    Given the breadth of the field of health analytics, this course will provide an overview of the development and rapid expansion of analytics in healthcare, major and emerging topical areas, and current issues related to research methods to improve human health. We will cover such topics as security concerns unique to the field, research design strategies, and the integration of epidemiologic and quality improvement methodologies to operationalize data for continuous improvement. Students will be introduced to the application of predictive analytics to healthcare. Students will understand factors impacting the delivery of quality and safe patient care and the application of data-driven methods to improve care at the healthcare system level, design approaches to answering a research question at the population level, become familiar with the application of data analytics to impacting care at the provider level through Clinical Decision Systems, and understand the process of a Clinical Trial.
  - [Data Visualization Techniques](#accordion-tab-5) 

    In today’s data driven enterprise, data storytelling using effective visualization strategies is an essential skill for analytics practitioners in almost every field to explore and present data. This course focuses on modern data visualization technologies, tools, and techniques to convert raw data into actionable information. Modern data visualization tools are at the forefront of the “self-service analytics” architectures which are decentralizing analytics and breaking down IT bottlenecks for business experts. Moreover, with its foundations rooted in statistics, psychology, and computer science, data visualization shows you how to better understand the data, present clear evidence of your findings to your intended audience and tell engaging data stories through charts and graphics. This course is designed to introduce data visualization as a medium of effective communication using strategic storytelling, and the basis for interactive information dashboards.
  - [Digital Marketing Analytics in Theory and Practice](#accordion-tab-6) 

    Successfully marketing brands today requires a well-balanced blend of art and science. This course introduces students to the science of web analytics while casting a keen eye toward the artful use of numbers found in the digital space. The goal is to provide marketers with the foundation needed to apply data analytics to real-world challenges they confront daily in their professional lives. Students will learn to identify the web analytic tool right for their specific needs; understand valid and reliable ways to collect, analyze, and visualize data from the web; and utilize data in decision making for their agencies, organizations or clients. By completing this course, students will gain an understanding of the motivations behind data collection and analysis methods used by marketing professionals; learn to evaluate and choose appropriate web analytics tools and techniques; understand frameworks and approaches to measuring consumers’ digital actions; earn familiarity with the unique measurement opportunities and challenges presented by New Media; gain hands-on, working knowledge of a step-by-step approach to planning, collecting, analyzing, and reporting data; utilize tools to collect data using today’s most important online techniques: performing bulk downloads, tapping APIs, and scraping webpages; and understand approaches to visualizing data effectively.
  - [Financial Analytics](#accordion-tab-7) 

    This course concentrates on the following topics: review of financial markets and assets traded on them; main characteristics of financial analytics: returns, yields, volatility; review of stochastic models of market price and their statistical representations; concept of arbitrage, elements of arbitrage pricing approach; principles of volatility analyses, implied vs. realized volatility; correlation, cointegration and other relationships between various financial assets; market risk analytics and management of portfolios of financial assets. The course puts special emphasis on covering main steps of building analytics from visualizing data and building intuition about their structure and patterns to selecting appropriate statistical method to interpretation of the results and building analytical models. Topics are illustrated by data analysis projects using R. Basic familiarity with R is a requirement.
  - [Generative AI Principles](#accordion-tab-8) 

    This course dives into the realm of Generative AI, offering a comprehensive look into the world of Large Language Models (LLMs), image generation techniques, and the fusion of vision and text through multimodal models. Drawing from core concepts in neural networks, transformers, and advanced techniques such as prompt engineering, vision prompting, and multimodality representation, students will explore the capabilities, applications, and ethical considerations of generative models. This course culminates in hands-on projects, allowing participants to apply theory to practical scenarios.
  - [Machine Learning Operations](#accordion-tab-9) 

    The objective of this course is two-fold: first, to understand what Machine Learning Operations (MLOps) is and why it is a key component in enterprise production deployment of machine learning projects, and second, to expose students to software engineering, model engineering and state-of-the-art deployment engineering with hands-on platform and tools experience. This course crosses the chasm that separates machine learning projects/experiments and enterprise production deployment. It covers three pillars in MLOps: software engineering such as software architecture, Continuous Integration/Continuous Delivery and data versioning; model engineering such as AutoML and A/B experimentation; and deployment engineering such as docker containers and model monitoring. The course focuses on best practices in the industry that are critical to enterprise production deployment of machine learning projects. Having completed this course, a student understands the machine learning lifecycle and what it takes to go from ideation to operationalization in an enterprise environment. Furthermore, students get exposure to state-of-the-art MLOps platforms such as allegro, xpresso, Dataiku, LityxIQ, DataRobot, AWS Sagemaker, and technologies such as gitHub, Jenkins, slack, docker, and kubernetes.
  - [Natural Language Processing and Cognitive Computing](#accordion-tab-10) 

    Extracting actionable insights from unstructured text and designing cognitive applications have become significant areas of application for analytics. Students in this course will learn foundations of natural language processing, including: concept extraction; text summarization and topic modeling; part of speech tagging; named entity recognition; semantic roles and sentiment analysis. For advanced NLP applications, we will focus on feature extraction from unstructured text, including word and paragraph embedding and representing words and paragraphs as vectors. For cognitive analytics section of the course, students will practice designing question answering systems with intent classification, semantic knowledge extraction and reasoning under uncertainty. Students will gain hands-on expertise applying Python for text analysis tasks, as well as practice with multiple IBM Watson services, including: Watson Discovery, Watson Conversation, Watson Natural Language Classification and Watson Natural Language Understanding.
  - [Real Time Intelligent Systems](#accordion-tab-11) 

    Developing end-to-end automation and intelligent systems is now the most advanced area of application for analytics. Building such systems requires proficiency in programming, understanding of computer systems, as well as knowledge of related analytical methodologies, which are the skills that this course aims to teach to students. The course focuses on python and is tailored for students with basic programming knowledge in python. The course is partially project based. During the first three sessions, we will review basic python concepts and then learn more advanced python and the ways to use python to handle large data flows. The later sessions are project based and will focus on developing end-to-end analytical solutions in the following areas: Finance and trading, blockchains and crypto-currencies, image recognition, and video surveillance systems.
  - [Reinforcement Learning](#accordion-tab-12) 

    This course is an introduction to reinforcement learning, also known as neuro-dynamic programming. It discusses basic and advanced concepts in reinforcement learning and provides several practical applications. Reinforcement learning refers to a system or agent interacting with an environment and learning how to behave optimally in such an environment. An environment typically includes time, actions, states, uncertainty and rewards. Reinforcement learning combines neural networks and dynamic programming to find an optimal behavior or policy of the system or agent in a complex environment setting. Neural network approximations are used to circumvent the well-known ‘curse of dimensionality’ which has been a barrier to solving many practical applications. Dynamic programming is the key learning mechanism that the system or the agent uses to interact with the environment and improve its performance. Students will master key learning techniques and will become proficient in applying these techniques to complex stochastic decision processes and intelligent control.
  - [Supply Chain Optimization](#accordion-tab-13) 

    “Big Data” continues to grow exponentially in our large-scale transactional world where 100,000s of SKUs and millions of customers are interacting with 1:1 offers that include differential pricing, shipping timing/costs and even made to order “custom” product configurations. These consumer behaviors are quickly advancing the availability of new data and techniques within the discipline of Data Science. This elective course will give students the opportunity to apply their skills in data visualization, data mining tools, predictive modeling, and advanced optimization techniques to address Supply Chain challenges. The course focuses on the use of Advanced Predictive Modeling, Machine Learning, AI and other Data Science insight and activation tools to automate and optimize the performance of the Supply Chain. Students will also learn how to optimize the performance of the Supply Chain from the lens of multiple related disciplines including: Sales Forecasting, Warehousing/Inventory Management, Promotion, Pricing, Logistics Network Optimization, Freight Cost Management, Manufacturing, Retail POS Information, Ecommerce, Consumer Data, and Product Design/Packaging. After completing this course, you will be prepared to work in any of the numerous specialty areas possible in the world of Supply Chain Management.





PRINTING: 011_tuition-fees-aid.md






# Source: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/tuition-fees-aid/
# Scraped: 2025-07-27

## MS in Applied Data Science facet-arrow-down

- [Overview](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/)
- [In-Person Program](https://datascience.uchicago.edu/education/masters-programs/in-person-program/)
- [Online Program](https://datascience.uchicago.edu/education/masters-programs/online-program/)
- [Capstone Projects](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)
- [Course Progressions](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/course-progressions/)
- [How to Apply](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/how-to-apply/)
- [Events & Deadlines](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/events-deadlines/)
- [Tuition, Fees, & Aid](https://datascience.uchicago.edu/education/tuition-fees-aid/)
- [Our Students](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/our-students/)
- [Faculty, Instructors, Staff](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/)
- [FAQs](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/)
- [Explore the MS-ADS Campus](https://datascience.uchicago.edu/explore-the-ms-ads-campus/)
- [Career Outcomes](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/career-outcomes/)
- [Get In Touch](https://apply-psd.uchicago.edu/register/?id=ef0bc7e7-7b6a-4888-92e1-0574384e9b9c&amp)

# Program Tuition

Tuition and course fees displayed are for Summer 2025, Autumn 2025, Winter 2026, and Spring 2026. Visit the [Office of the University Bursar website](https://bursar.uchicago.edu/tuition-and-fees/tuition-and-fees-2025-26/tuition-and-fees-2025-26-physical-sciences-division) for information on quarterly and occasional fees, as well as information about tuition rates.

- Tuition for the MS in Applied Data Science program: $6,384 per course/$76,608 total tuition\*\*
- Program Enrollment Deposit (non-refundable): $1,500 (credited toward your first quarter’s tuition balance)

\* Enrolling in a program of study includes additional costs beyond the tuition listed here. For a full list of direct and indirect expenses associated with the cost of attendance, please visit the [Cost of Attendance](https://financialaid.uchicago.edu/graduate/costs/cost-of-attendance/) page of the Graduate Financial Aid Office website.

Note: Tuition is expected to increase 3-7% per year.

*\*\* The program awards merit-based scholarships.*

# Noncredit Courses

- Career Seminar (required unless eligible to waive): $0
- Introduction to Statistical Concepts (optional): $0
- R Foundational Course (optional): $0
- Python Foundational Course (optional): $0
- Advanced Linear Algebra for Machine Learning (optional): $0

## The Data Science Institute Scholarship, MS in Applied Data Science Alumni Scholarship

The MS in Applied Data Science program offers partial tuition scholarships to top applicants. These scholarships do not require a separate application but it is recommended that candidates submit their applications ahead of the early deadline to maximize their chances of securing a scholarship.

### Other Scholarships

Students are encouraged to investigate scholarships offered through various civic and professional organizations, foundations and state agencies. One place to search for scholarships is the [financial aid information web page](https://finaid.org/) sponsored by the National Association of Student Aid Administration.

## Financial Aid

Merit scholarships are available for eligible applicants. Once you apply to the program, you will be automatically considered for a scholarship. Early applications are highly encouraged.

In addition, if you are interested in financial aid to help finance your graduate education, once admitted, you will work with the University of Chicago’s Student Loan Administration office.

To understand the types of financial assistance available and to gather additional information, please visit the [Graduate Aid page](https://financialaid.uchicago.edu/graduate). For international student funding options, please visit the [International Student Funding Options page](https://internationalaffairs.uchicago.edu/fundingoptions).

To find out if you are eligible for federal loans, please visit our [Aid Eligibility page](https://financialaid.uchicago.edu/graduate/types-aid/aid-eligibility-criteria). We have administrative systems in place to assist students in applying for financial aid upon admission to programs. Please visit our Tuition, Financing, and Billing page for more information.

For information and assistance with employer tuition benefits, please visit the [website of the Bursar](http://bursar.uchicago.edu/).

If you are interested in financial aid through student employment, please visit the [Student Employment website](https://studentemployment.uchicago.edu/).

Visit our [FAQ](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/) page for additional information.

## Start Your Application

The [In-Person](https://datascience.uchicago.edu/education/masters-programs/in-person-program/) and the [Online](https://datascience.uchicago.edu/education/masters-programs/online-program/) program admit full- and part-time students for entrance in autumn quarter.





PRINTING: 012_%20.md






# Source: https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/online-program/%20
# Scraped: 2025-07-27

## MS in Applied Data Science facet-arrow-down

- [Overview](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/)
- [In-Person Program](https://datascience.uchicago.edu/education/masters-programs/in-person-program/)
- [Online Program](https://datascience.uchicago.edu/education/masters-programs/online-program/)
- [Capstone Projects](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)
- [Course Progressions](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/course-progressions/)
- [How to Apply](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/how-to-apply/)
- [Events & Deadlines](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/events-deadlines/)
- [Tuition, Fees, & Aid](https://datascience.uchicago.edu/education/tuition-fees-aid/)
- [Our Students](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/our-students/)
- [Faculty, Instructors, Staff](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/)
- [FAQs](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/)
- [Explore the MS-ADS Campus](https://datascience.uchicago.edu/explore-the-ms-ads-campus/)
- [Career Outcomes](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/career-outcomes/)
- [Get In Touch](https://apply-psd.uchicago.edu/register/?id=ef0bc7e7-7b6a-4888-92e1-0574384e9b9c&amp)

## Rigor Meets Flexibility

You will benefit from [the same top faculty, bright peer group, and academic rigor](https://www.youtube.com/watch?v=9kHtZxp1RVg&list=PL0IrIAIuK93EonLgPKZ7oIcpt0p_j58vm&index=9) as the In-Person program. You may complete the program full- or part-time. Full-time and part-time students begin in [Autumn quarter.](https://www.uchicago.edu/en/education-and-research/academic-calendar) Please visit the [In-Person](https://datascience.uchicago.edu/education/masters-programs/in-person-program/) program page if you are interested in exploring those full- and part-time options for entrance Autumn quarter.

*Online Program Director, [Arnab Bose, PhD](https://datascience.uchicago.edu/people/arnab-bose-phd/)*

## Master Data Science, Master Your Career

Get ready for what’s next in data science with rigorous classes, expert instructors, leading-edge technology, and an [unparalleled network of industry professionals](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/). A [full-service team](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/) dedicated to the MS in Applied Data Science program are here to help you reach your career potential as a member of the University of Chicago community.

## Get in Touch

Loading…

Read More +
Show Less -

## Your Outcomes

You will be a top competitor among data science jobs and career paths as a result of this program. You will have access to a dedicated team of faculty and staff who are committed to helping you reach your potential–whether it’s landing a key internship, preparing for a promotion, or moving into your next job, the MS in Applied Data Science program is here to help. Among other resources, you will have access to tailored, in-house career services, academic advising, and opportunities to grow your network in person and virtually.

## Your Student Experience

There are unique opportunities to support you as an Online master’s student. You will have access to the same full-service team of student affairs, career services, and instructional services experts. Jointly with [faculty and instructors](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/), our Master’s in Applied Data Science team is committed to providing a supportive and enriching student experience. Our team is passionate about supporting students engaging in online education who might also be working full-time like you. Staff who lead our Online program have a background in instructional design and best practices for online learning.

## Your Access to Alumni

As an Online student, you will have access to program alumni through Immersion Weekend events and virtual career services programs–all of which broadens your network and supports your career advancement goals. Upon graduation, you will also join a prestigious group of [Physical Sciences Division](https://physicalsciences.uchicago.edu/about/alumni-friends/) and [UChicago alumni](https://alumniandfriends.uchicago.edu/s/directory) who live and work all over the world. Our Alumni Board that plans events and programs in Chicago and other U.S. cities.

## Your Time, Your Advancement

You will engage in weekly, synchronous (real-time) classes with your instructors and peers. Live lectures provide the right forum to ask questions, work with peers, and maximize understanding of key concepts.  Collaborative and interactive learning with your classmates is facilitated by advanced meeting technology during and beyond synchronous classes.

Synchronous classes are scheduled on weekday evenings at 6 pm, CT and on occasional Saturdays at 9 am, CT. Students admitted into the online program are expected to attend the live class sessions.

Each week also includes asynchronous content (learning at your own pace) to complete when it is convenient for you. These activities prepare you for synchronous classes through lecture videos, readings, and discussions. Flexibility allows you to work on data science projects, group collaborations, and assignments on your schedule.

## In-Person ‘Immersion Weekend’

You will have the opportunity to attend a unique, in-person ‘Immersion Weekend’ in Chicago in the Autumn quarter.

**During the Immersion Weekend, you will have the opportunity to**

- Network with industry leaders and [alumni](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/our-students/)
- Meet a vibrant and supportive community of like-minded students and faculty
- Get to know, bond, and network with peers from across the country
- Participate in interactive sessions, working to develop industry solutions
- Gain career insights and discuss the latest trends and challenges in AI/ML
- Hear from students about their experience in [Capstone Projects](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)
- Tour the [Data Science Institute](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/)

**Future Immersion Weekend dates are:**

- Autumn 2025 – Friday, September 26 – Saturday, September 27

We look forward to seeing you at the event!

#### Note for International Students

If you are coming from a non-US location for Immersion Weekend programming, you are encouraged to come to the U.S. in visitor/tourist status; using a B-1/B-2 visa or ESTA clearance depending on whether you hold citizenship from a Visa Waiver Country or not.

The immersion days are optional for students and do not carry any course enrollment for the time you are in Chicago. Please be clear when entering the U.S. that while you are a student in an online program at the University of Chicago, your visit for immersion weekend is not to attend credit-bearing classes, but to meet classmates and as a result you are coming to the U.S. as a visitor.

Read More +
Show Less -

## Curriculum

You will earn UChicago’s Master’s in Applied Data Science by successfully completing 12 courses (6 core, 4 elective, 2 Capstone) and our tailored Career Seminar\*.

Our rigorous curriculum is designed by and for data science innovators and leaders. Courses are reviewed annually to ensure the content keeps pace with the [rapidly evolving landscape of data science](https://datascience.uchicago.edu/news-events/news/).

You have the flexibility to pursue the [Master’s in Applied Data Science](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/) degree on a part- or full-time schedule. Part-time students enroll in two courses each quarter and take their courses in the evenings or on Saturdays. Full-time students take three courses per quarter. Some of their courses may be offered during the day.

### Foundational Courses (noncredit, optional)

Foundational noncredit courses are designed and taught by Master’s in Applied Data Science [faculty and instructors](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/). These optional courses—available at no additional cost— provide the basis for the rigorous Applied Data Science degree. Course content undergirds the theoretical, strategic, and practical data science studies you will encounter in the rest of the curriculum.  

Beginning in academic year 2024-25, all entering students will complete a required online Foundational Skills Assessment. This required assessment helps faculty and advisors understand how to best support you once you begin in the program. You may choose to opt-out of taking the Foundational Skill Assessments and instead register for all four Foundational courses. The four Foundational noncredit courses are listed below. Please note that Introduction to Statistical Concepts and R are considered pre-quarter courses and therefore take place during the 5 weeks leading up to your first quarter in the program. All Foundational courses are completed virtually for all students regardless of enrollment in the In-Person or Online Program.

Read More +
Show Less -

### Career Seminar (noncredit, required)

Increasingly, employers demand data scientists and analytics professionals who are not only technically excellent but also superior collaborators, reliable communicators, ethical problem-solvers, and more. To ensure that our graduates remain as top candidates in the job market, we provide tailored support through our required, multi-quarter Career Seminar. The Seminar is designed to counter-balance students’ time commitments in other courses. \*Students with significant, relevant full-time work experience may be eligible to waive this requirement.

### Core Courses (6)

You will complete 6 core courses toward your [Master’s in Applied Data Science](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/instructors-staff/) degree. Core courses allow you to build your theoretical data science knowledge and practice applying this theory to examine real-world business problems.

### Elective Courses (4)

You will complete 4 required electives toward your 12-course degree program. We continually add electives to evolve with the data science landscape. Past electives include Advanced Computer Vision with Deep Learning, Advanced Machine Learning and Artificial Intelligence, Bayesian Methods, Data Science for Algorithmic Marketing, Data Visualization Techniques, Digital Marketing Analytics in Theory and Practice, Financial Analytics, Health Analytics, Machine Learning Operations, Natural Language Processing and Cognitive Computing, Real Time Intelligent Systems, Reinforcement Learning, Supply Chain Optimization.

### Capstone (2)

The required [Capstone Project](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/) is completed over 2 quarters and covers research design, implementation, and writing. Full-time students start their [Capstone Project](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/) in their third quarter. Part-time students generally begin the [Capstone Project](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/capstone-projects/) two quarters before their projected [graduation](https://convocation.uchicago.edu/) quarter. Students choose among industry- and research-focused projects.

### Additional Resources

- [Upcoming Events & Deadlines](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/events-deadlines/)
- [FAQs](https://datascience.uchicago.edu/education/masters-programs/ms-in-applied-data-science/faqs/)
- [Tuition, Fees, & Aid](https://datascience.uchicago.edu/education/tuition-fees-aid/)

- [Noncredit Courses](#)

  - [Career Seminar (Seminar, required)](#accordion-tab-0) 

    The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.
  - [Introduction to Statistical Concepts (Foundational, optional)](#accordion-tab-1) 

    This Foundational, optional course is held in the 5 weeks leading up to the start of your first quarter. This course provides general exposure to basic statistical concepts that are necessary for students to understand the content presented in more advanced courses in the program. 0 units, no cost.
  - [R for Data Science (Foundational, optional)](#accordion-tab-2) 

    This Foundational, optional course is held in the 5 weeks leading up to the start of your first quarter. This course is an introduction to the essential concepts and techniques for the statistical computing language R. 0 units, no cost.
  - [Python for Data Science (Foundational, optional)](#accordion-tab-3) 

    This Foundational, optional course is held concurrently with the first five weeks of your first quarter in the program. This course in Python starts with an introduction to the Python programming language basic syntax and environment. 0 units, no cost.
  - [Advanced Linear Algebra for Machine Learning (Foundational, optional)](#accordion-tab-4) 

    This Foundational, optional course is held concurrently with the second five weeks of your first quarter in the program. The advanced linear algebra course is focused on the theoretical concepts and real-life applications of linear algebra for machine learning. 0 units, no cost.
  - [Brush up on the Basics (Optional resource)](#accordion-tab-5) 

    If you would like to gauge your preparation in these Foundational course topics, we recommend specific Coursera courses that cover very similar topics.

    We have identified four Coursera courses which cover very similar topics. You can review the Coursera curricula to see if you are already well-prepared, or if you like, study their materials to brush up on some or all of these topics.

    [Mathematics for Machine Learning: Linear Algebra (offered by University College London)](https://www.coursera.org/learn/linear-algebra-machine-learning)

    [Basis Data Descriptors, Statistical Distributions, and Application to Business Decisions (offered by Rice University)](https://www.coursera.org/learn/descriptive-statistics-statistical-distributions-business-application?specialization=business-statistics-analysis)

    [Python for Data Science, AI, & Development (offered by IBM)](https://www.coursera.org/learn/python-for-applied-data-science-ai?specialization=ibm-data-science)

    [Data Analysis with R Programming (offered by Google)](https://www.coursera.org/learn/data-analysis-r)
- [Core Courses](#)

  - [Time Series Analysis and Forecasting](#accordion-tab-0) 

    Time Series Analysis is a science as well as the art of making rational predictions based on previous records. It is widely used in various fields in today’s business settings.
  - [Statistical Models for Data Science](#accordion-tab-1) 

    In a traditional linear model, the observed response follows a normal distribution, and the expected response value is a linear combination of the predictors.  Since Carl Friedrich Gauss (1777-1855) and Adrien-Marie Legendre (1752-1833) created this linear model framework in the early 1800s, the “Linear Normal” assumption has been the norm in statistics/data science for almost two centuries.  New methods based on probability distributions other than Gaussian appeared only in the second half of the twentieth century. These methods allowed working with variables that span a broader variety of domains and probability distributions. Besides, methods for the analysis of general associations were developed that are different from the Pearson correlation.
  - [Machine Learning I](#accordion-tab-2) 

    This course is aimed at providing students an introduction to machine learning with data mining techniques and algorithms. It gives a rigorous methodological foundation in analytical and software tools to successfully undertake projects in Data Science. Students are exposed to concepts of exploratory analyses for uncovering and detecting patterns in multivariate data, hypothesizing and detecting relationships among variables, conducting confirmatory analyses, and building models for predictive and descriptive purposes. It will present predictive modeling in the context of balancing predictive and descriptive accuracies.
  - [Machine Learning II](#accordion-tab-3) 

    The objective of this course is three-folds–first, to extend student understanding of predictive modeling with machine learning concepts and methodologies from Machine Learning 1 into the realm of Deep Learning and Generative AI. Second, to develop the ability to apply those concepts and methodologies to diverse practical applications, evaluate the results and recommend the next best action. Third, to discuss and understand state-of-the machine learning and deep learning research and development and their applications.
  - [Data Engineering Platforms for Analytics or Big Data and Cloud Computing](#accordion-tab-4) 

    Data Engineering Platforms teaches effective data engineering—an essential first step in building an analytics-driven competitive advantage in the market.

    Big Data and Cloud Computing teaches students how to approach big data and large-scale machine learning applications. There is no single definition of big data and multiple emerging software packages exist to work with it, and we will cover the most popular approaches.
  - [Leadership and Consulting for Data Science](#accordion-tab-5) 

    The Leadership and Consulting for Data Scientist course is focused on:

    • Learning techniques and proven methods to effectively grasp the business domain including organizational dynamics of consultancies and client organizations  
    • Developing relevant solutions to enterprise problems using the sampling methods, traditional statistical techniques and modern machine learning models that deliver value to the organization  
    • Practicing successful project delivery through effective data discovery, influential team membership and leadership, project management, and communication at every stage

    This course will not only make you a better data scientist; it will make you and your analyses more approachable, more persuasive, and ultimately more successful.
  - [Data Science Capstone Project](#accordion-tab-6) 

    The required Capstone Project is completed over two quarters and covers research design, implementation, and writing. Full-time students start their capstone project in their third quarter. Part-time students generally begin the capstone project in their fifth quarter.
- [Sample Elective Courses](#)

  - [Advanced Computer Vision with Deep Learning](#accordion-tab-0) 

    Computer vision is the field of computer science that focuses on creating digital systems that can process, analyze, and make sense of visual data in the same way that humans do. Deep learning is a subset of machine learning and a branch of Artificial Intelligence (AI). It involves the training, deployment, and application of large complex neural network architectures to solve cutting-edge problems. Deep Learning has become the primary approach for solving cognitive problems such as Computer Vision and Natural Language Processing (NLP) and has had a massive impact on various industries such as healthcare, retail, automotive, industrial automation, and agriculture. This course will enable students to build Deep Learning models and apply them to computer vision tasks such as object recognition, detection, and segmentation. Students will gain an in-depth understanding of the Deep Learning model development process, tools, and frameworks. Although the focus of the course will primarily be computer vision, students will work on both image and nonimage datasets during class exercises and assignments. Students will gain hands-on experience in popular libraries such as Tensorflow, Keras, and PyTorch. Students will also learn to apply state of the art models such as ResNet, EfficientNet, RCNNs, YOLO, Vision Transformers, etc. for computer vision and work on datasets such as CIFAR, ImageNet, MS COCO, and MPII Human Poses.
  - [Advanced Machine Learning and Artificial Intelligence](#accordion-tab-1) 

    Since the era of big data started, challenges associated with data analysis have grown significantly in different directions: First, the technological infrastructure had to be developed that can hold and process large amounts of data from different sources and of multiple not always well formalized formats. Second, data analysis methods had to be reviewed, selected and modified to work in distributed computational environments like combinations of in-house clusters of servers and cloud. But the biggest challenge of all is learning to think differently in order to ask new types of questions that could not be answered by analyses of less complex data streams with less complex technological infrastructure. In recent years significant progress has been achieved in creating technological ecosystems for big data analysis. Innovative technologies such as open source projects MapReduce, Hadoop, Spark, Storm, Kafka, TensorFlow, H2O, etc. allowed us to look at depths of data unseen before. We now have a growing number of sources and educational courses introducing these new tools. However, developing new data analysis methods appropriate to these new data ecosystems is more difficult than it appears.
  - [Bayesian Machine Learning with Generative AI Applications](#accordion-tab-2) 

    This course provides a strong theoretical and practical skillset for probabilistic machine learning applications. Bayesian inference and modeling methods are important for several areas including prediction, decision making, and risk assessment where modeling the uncertainty is needed. The course begins with an introduction to Bayesian statistical analysis, covering the foundations of Bayesian inference and the application of Bayes’ theorem for statistical inference. We then introduce Bayesian networks, which offer a powerful graphical tool for modeling complex systems and making probabilistic inferences. The course then advances to cover more sophisticated topics such as Markov Chain Monte Carlo (MCMC) methods for sampling from complex probability distributions, hierarchical models, and model selection techniques. The final three weeks are dedicated to cutting-edge methodologies like Generative Deep Learning, Variational Autoencoders, and Bayesian Neural Networks, all rooted in Bayesian Machine Learning. Upon completion, students will be equipped to apply Bayesian methods to a wide range of real-world problems in fields such as engineering, business, finance, and public policy, addressing challenges like missing data or training AI models that are able to say ‘I don’t know’.
  - [Data Science for Algorithmic Marketing](#accordion-tab-3) 

    This course focuses on marketing science methods and algorithms for undertaking competitive analysis in the digital landscape: market segmentation, mining databases for effective digital marketing, design of new digital and traditional products, forecasting sales and product diffusion, real time product positioning, intra omni-channel optimization and inter omni-channel resource allocation, and pricing across both omni-channel marketing effectiveness and ROI. The course will use a combination of lecture, in-class discussions, group assignments, and a final group project. The course lays special emphasis on algorithms. Hence it draws heavily from the fields of optimization, machine-learning based recommendation systems, association rules, consumer choice models, Bayesian estimation, experimentation and analysis of covariance, advanced visualization techniques for mapping brand perceptions, and analysis of social media data using advanced NLP techniques.
  - [Data Science for Healthcare](#accordion-tab-4) 

    Given the breadth of the field of health analytics, this course will provide an overview of the development and rapid expansion of analytics in healthcare, major and emerging topical areas, and current issues related to research methods to improve human health. We will cover such topics as security concerns unique to the field, research design strategies, and the integration of epidemiologic and quality improvement methodologies to operationalize data for continuous improvement. Students will be introduced to the application of predictive analytics to healthcare. Students will understand factors impacting the delivery of quality and safe patient care and the application of data-driven methods to improve care at the healthcare system level, design approaches to answering a research question at the population level, become familiar with the application of data analytics to impacting care at the provider level through Clinical Decision Systems, and understand the process of a Clinical Trial.
  - [Data Visualization Techniques](#accordion-tab-5) 

    In today’s data driven enterprise, data storytelling using effective visualization strategies is an essential skill for analytics practitioners in almost every field to explore and present data. This course focuses on modern data visualization technologies, tools, and techniques to convert raw data into actionable information. Modern data visualization tools are at the forefront of the “self-service analytics” architectures which are decentralizing analytics and breaking down IT bottlenecks for business experts. Moreover, with its foundations rooted in statistics, psychology, and computer science, data visualization shows you how to better understand the data, present clear evidence of your findings to your intended audience and tell engaging data stories through charts and graphics. This course is designed to introduce data visualization as a medium of effective communication using strategic storytelling, and the basis for interactive information dashboards.
  - [Digital Marketing Analytics in Theory and Practice](#accordion-tab-6) 

    Successfully marketing brands today requires a well-balanced blend of art and science. This course introduces students to the science of web analytics while casting a keen eye toward the artful use of numbers found in the digital space. The goal is to provide marketers with the foundation needed to apply data analytics to real-world challenges they confront daily in their professional lives. Students will learn to identify the web analytic tool right for their specific needs; understand valid and reliable ways to collect, analyze, and visualize data from the web; and utilize data in decision making for their agencies, organizations or clients. By completing this course, students will gain an understanding of the motivations behind data collection and analysis methods used by marketing professionals; learn to evaluate and choose appropriate web analytics tools and techniques; understand frameworks and approaches to measuring consumers’ digital actions; earn familiarity with the unique measurement opportunities and challenges presented by New Media; gain hands-on, working knowledge of a step-by-step approach to planning, collecting, analyzing, and reporting data; utilize tools to collect data using today’s most important online techniques: performing bulk downloads, tapping APIs, and scraping webpages; and understand approaches to visualizing data effectively.
  - [Financial Analytics](#accordion-tab-7) 

    This course concentrates on the following topics: review of financial markets and assets traded on them; main characteristics of financial analytics: returns, yields, volatility; review of stochastic models of market price and their statistical representations; concept of arbitrage, elements of arbitrage pricing approach; principles of volatility analyses, implied vs. realized volatility; correlation, cointegration and other relationships between various financial assets; market risk analytics and management of portfolios of financial assets. The course puts special emphasis on covering main steps of building analytics from visualizing data and building intuition about their structure and patterns to selecting appropriate statistical method to interpretation of the results and building analytical models. Topics are illustrated by data analysis projects using R. Basic familiarity with R is a requirement.
  - [Generative AI Principles](#accordion-tab-8) 

    This course dives into the realm of Generative AI, offering a comprehensive look into the world of Large Language Models (LLMs), image generation techniques, and the fusion of vision and text through multimodal models. Drawing from core concepts in neural networks, transformers, and advanced techniques such as prompt engineering, vision prompting, and multimodality representation, students will explore the capabilities, applications, and ethical considerations of generative models. This course culminates in hands-on projects, allowing participants to apply theory to practical scenarios.
  - [Machine Learning Operations](#accordion-tab-9) 

    The objective of this course is two-fold: first, to understand what Machine Learning Operations (MLOps) is and why it is a key component in enterprise production deployment of machine learning projects, and second, to expose students to software engineering, model engineering and state-of-the-art deployment engineering with hands-on platform and tools experience. This course crosses the chasm that separates machine learning projects/experiments and enterprise production deployment. It covers three pillars in MLOps: software engineering such as software architecture, Continuous Integration/Continuous Delivery and data versioning; model engineering such as AutoML and A/B experimentation; and deployment engineering such as docker containers and model monitoring. The course focuses on best practices in the industry that are critical to enterprise production deployment of machine learning projects. Having completed this course, a student understands the machine learning lifecycle and what it takes to go from ideation to operationalization in an enterprise environment. Furthermore, students get exposure to state-of-the-art MLOps platforms such as allegro, xpresso, Dataiku, LityxIQ, DataRobot, AWS Sagemaker, and technologies such as gitHub, Jenkins, slack, docker, and kubernetes.
  - [Natural Language Processing and Cognitive Computing](#accordion-tab-10) 

    Extracting actionable insights from unstructured text and designing cognitive applications have become significant areas of application for analytics. Students in this course will learn foundations of natural language processing, including: concept extraction; text summarization and topic modeling; part of speech tagging; named entity recognition; semantic roles and sentiment analysis. For advanced NLP applications, we will focus on feature extraction from unstructured text, including word and paragraph embedding and representing words and paragraphs as vectors. For cognitive analytics section of the course, students will practice designing question answering systems with intent classification, semantic knowledge extraction and reasoning under uncertainty. Students will gain hands-on expertise applying Python for text analysis tasks, as well as practice with multiple IBM Watson services, including: Watson Discovery, Watson Conversation, Watson Natural Language Classification and Watson Natural Language Understanding.
  - [Real Time Intelligent Systems](#accordion-tab-11) 

    Developing end-to-end automation and intelligent systems is now the most advanced area of application for analytics. Building such systems requires proficiency in programming, understanding of computer systems, as well as knowledge of related analytical methodologies, which are the skills that this course aims to teach to students. The course focuses on python and is tailored for students with basic programming knowledge in python. The course is partially project based. During the first three sessions, we will review basic python concepts and then learn more advanced python and the ways to use python to handle large data flows. The later sessions are project based and will focus on developing end-to-end analytical solutions in the following areas: Finance and trading, blockchains and crypto-currencies, image recognition, and video surveillance systems.
  - [Reinforcement Learning](#accordion-tab-12) 

    This course is an introduction to reinforcement learning, also known as neuro-dynamic programming. It discusses basic and advanced concepts in reinforcement learning and provides several practical applications. Reinforcement learning refers to a system or agent interacting with an environment and learning how to behave optimally in such an environment. An environment typically includes time, actions, states, uncertainty and rewards. Reinforcement learning combines neural networks and dynamic programming to find an optimal behavior or policy of the system or agent in a complex environment setting. Neural network approximations are used to circumvent the well-known ‘curse of dimensionality’ which has been a barrier to solving many practical applications. Dynamic programming is the key learning mechanism that the system or the agent uses to interact with the environment and improve its performance. Students will master key learning techniques and will become proficient in applying these techniques to complex stochastic decision processes and intelligent control.
  - [Supply Chain Optimization](#accordion-tab-13) 

    “Big Data” continues to grow exponentially in our large-scale transactional world where 100,000s of SKUs and millions of customers are interacting with 1:1 offers that include differential pricing, shipping timing/costs and even made to order “custom” product configurations. These consumer behaviors are quickly advancing the availability of new data and techniques within the discipline of Data Science. This elective course will give students the opportunity to apply their skills in data visualization, data mining tools, predictive modeling, and advanced optimization techniques to address Supply Chain challenges. The course focuses on the use of Advanced Predictive Modeling, Machine Learning, AI and other Data Science insight and activation tools to automate and optimize the performance of the Supply Chain. Students will also learn how to optimize the performance of the Supply Chain from the lens of multiple related disciplines including: Sales Forecasting, Warehousing/Inventory Management, Promotion, Pricing, Logistics Network Optimization, Freight Cost Management, Manufacturing, Retail POS Information, Ecommerce, Consumer Data, and Product Design/Packaging. After completing this course, you will be prepared to work in any of the numerous specialty areas possible in the world of Supply Chain Management.